# BiLSTM advanced model

In this notebook we trained the BiLSTM advanced model with Keras Tuner using the training and validation set created during preprocessing.

## Initialization

On Colab upload manually the file `dataset.zip`

In [1]:
!unzip -o dataset.zip

Archive:  dataset.zip
  inflating: x_train.npy             
  inflating: y_train.npy             
  inflating: x_train_ov.npy          
  inflating: y_train_ov.npy          
  inflating: x_val.npy               
  inflating: y_val.npy               


### Import libraries

In [2]:
import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.rc('font', size=16) 
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
import sklearn.preprocessing as sklp
import warnings
import logging

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

2.9.2


In [3]:
!pip install -q -U keras-tuner
import keras_tuner as kt

     |████████████████████████████████| 135 kB 5.0 MB/s 
     |████████████████████████████████| 1.6 MB 40.3 MB/s 


### Set seed for reproducibility

In [4]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

### Load dataset

In [5]:
x_train = np.load("x_train.npy")
y_train = np.load("y_train.npy")

x_val = np.load("x_val.npy")
y_val = np.load("y_val.npy")

x_train.shape, x_val.shape, y_train.shape, y_val.shape

((1938, 36, 6), (491, 36, 6), (1938, 12), (491, 12))

## Model

In [6]:
input_shape = (36, 6)
classes = 12
batch_size = 32
epochs = 100

### Custom Augmentation Layer

In [7]:
class CustomAugmentationLayer(tfkl.Layer):
    def __init__(self, **kwargs):
        super(CustomAugmentationLayer, self).__init__(**kwargs)

    def random_float(self, start, end, precision):
      return round(random.uniform(start, end), precision)

    def custom_scaling(self, x):
      return x * (1 - self.random_float(0, 0.2, 2))

    def custom_shift(self, x):
      return x + self.random_float(-0.2, 0.2, 2)

    def call(self, inputs, training=None):
        if training:
  
          if random.random() < 0.5:
            x = self.custom_scaling(inputs)

          if random.random() < 0.5:
            x = self.custom_shift(inputs)

        return inputs

### Definition

In [8]:
def build_model(hp):
  model = tfk.models.Sequential(name='model_bi')
  model.add(tfkl.Input(input_shape, name='bi_input'))

  model.add(tfkl.Normalization())
  model.add(CustomAugmentationLayer(name='custom_augmentation_layer'))

  # Feature extractor
  hp_units_1 = hp.Int('units_1', min_value=64, max_value=512, step=32)
  model.add(tfkl.Bidirectional(tfkl.LSTM(hp_units_1, return_sequences=True, name='bi_lstm_1')))

  hp_units_2 = hp.Int('units_2', min_value=64, max_value=512, step=32)
  model.add(tfkl.Bidirectional(tfkl.LSTM(hp_units_2, name='bi_lstm_2')))

  hp_dropout = hp.Float('dropout', min_value=0, max_value=0.4, step=0.1)
  model.add(tfkl.Dropout(hp_dropout, seed=seed, name='bi_dropout'))

  # Classifier
  hp_dense_1_units = hp.Int('dense_1_units', min_value=64, max_value=512, step=64)
  model.add(tfkl.Dense(hp_dense_1_units, activation='relu', name='bi_dense_1'))

  hp_dense_2_units = hp.Int('dense_2_units', min_value=64, max_value=512, step=64)
  model.add(tfkl.Dense(hp_dense_2_units, activation='relu', name='bi_dense_2'))

  hp_dense_dropout = hp.Float('dense_dropout', min_value=0, max_value=0.4, step=0.1)
  model.add(tfkl.Dropout(hp_dense_dropout, seed=seed, name='bi_dense_dropout'))

  # Output
  model.add(tfkl.Dense(classes, activation='softmax', name='bi_output'))

  # Compile
  model.compile(optimizer=tfk.optimizers.Adam(), loss=tfk.losses.CategoricalCrossentropy(), metrics=['accuracy'])

  return model

### Tuner

In [9]:
!rm -rf tuner

In [10]:
tuner = kt.Hyperband(
  build_model,
  objective='val_loss',
  max_epochs=epochs+20,
  factor=3,
  seed=seed,
  directory='tuner',
  project_name='tuner'
)

In [11]:
# Search for the best model
tuner.search(
  x=x_train, 
  y=y_train, 
  epochs=epochs, 
  batch_size=batch_size, 
  validation_data=(x_val, y_val),
  callbacks = [
    tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=15, factor=0.1, min_lr=1e-5),
    tfk.callbacks.EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)
  ]
)

# Get the best hyperparameters
best_hp = tuner.get_best_hyperparameters()[0]

# Get the best model
best_model = tuner.get_best_models()[0]

# Print summary of best model
tuner.results_summary(1)

Trial 254 Complete [00h 01m 01s]
val_loss: 1.0532370805740356

Best val_loss So Far: 0.993152379989624
Total elapsed time: 01h 26m 33s
Results summary
Results in tuner/tuner
Showing 1 best trials
Trial summary
Hyperparameters:
units_1: 352
units_2: 416
dropout: 0.4
dense_1_units: 128
dense_2_units: 384
dense_dropout: 0.0
tuner/epochs: 120
tuner/initial_epoch: 40
tuner/bracket: 4
tuner/round: 4
tuner/trial_id: 0144
Score: 0.993152379989624


### Results

In [12]:
train_loss, train_accuracy = best_model.evaluate(x_train, y_train)
val_loss, val_accuracy = best_model.evaluate(x_val, y_val)
print("train_loss:  %.4f  train_accuracy: %.4f" % (train_loss, train_accuracy))
print("val_loss:    %.4f  val_accuracy:   %.4f" % (val_loss, val_accuracy))

16/16 [==============================] - 0s 10ms/step - loss: 0.9932 - accuracy: 0.6701
train_loss:  0.5773  train_accuracy: 0.8173
val_loss:    0.9932  val_accuracy:   0.6701


### Save

In [13]:
best_model.save("model")
!zip -r model.zip model

  adding: model/ (stored 0%)
  adding: model/saved_model.pb (deflated 91%)
  adding: model/variables/ (stored 0%)
  adding: model/variables/variables.data-00000-of-00001 (deflated 7%)
  adding: model/variables/variables.index (deflated 70%)
  adding: model/assets/ (stored 0%)
  adding: model/keras_metadata.pb (deflated 92%)
